In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Criar dataset simulado
np.random.seed(44)
n = 800

# Variáveis de entrada (features)
dados_sensor = pd.DataFrame({
    "temp_op": np.random.normal(70, 10, n),
    "vibracao": np.random.normal(5, 2, n),
    "corrente": np.random.normal(1.5, 0.5, n),
    "material": np.random.choice(["A", "B", "C"], n, p=[0.5, 0.3, 0.2]),
})

# Criar a variável-alvo (rótulo) - 1 = Defeituoso, 0 = Normal
# A regra simula condições que levam ao defeito.
dados_sensor["defeito"] = (
    (dados_sensor["vibracao"] > 7) |
    (dados_sensor["temp_op"] < 50) |
    ((dados_sensor["material"] == "C") & (dados_sensor["corrente"] < 1.0))
).astype(int)

# Pré-processar os dados
# Codificação One-Hot para a variável categórica 'material'
dados_sensor = pd.get_dummies(dados_sensor, columns=['material'], drop_first=True)

# Variáveis Preditivas (X) e Variável Alvo (y)
X = dados_sensor.drop("defeito", axis=1)
y = dados_sensor["defeito"]

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44)

# Modelos a serem comparados
modelos = {
    # 'balanced' para lidar com possível desbalanceamento das classes
    "Regressão Logística": LogisticRegression(solver='liblinear', random_state=44, class_weight='balanced'),
    "K-Nearest Neighbors (KNN)": KNeighborsClassifier(n_neighbors=5)
}

resultados_sensor = {}

# Treinar e avaliar os modelos
print("Detecção de Defeitos em Sensores de Linha de Produção\n")

for nome, modelo in modelos.items():
    # Treinamento do modelo
    modelo.fit(X_train, y_train)

    # Previsão
    y_pred = modelo.predict(X_test)

    # Cálculo das métricas
    acuracia = accuracy_score(y_test, y_pred)
    precisao = precision_score(y_test, y_pred, zero_division=0)
    # Recall (Revocação) é crucial para minimizar Falsos Negativos (FN)
    revocacao = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    resultados_sensor[nome] = {
        "Acurácia": acuracia,
        "Precisão": precisao,
        "Recall": revocacao,
        "F1-Score": f1
    }

    print(f"--- Modelo: {nome} ---")

    # Matriz de Confusão: [[VN, FP], [FN, VP]]
    matriz = confusion_matrix(y_test, y_pred)
    print("Matriz de Confusão (0=Normal, 1=Defeituoso):")
    print(matriz)
    # FN são os defeituosos que o modelo classificou como normais (perigo!)
    print(f"  Falsos Negativos (FN): {matriz[1, 0]}")

    # Exibição das métricas
    print(f"\nAcurácia: {acuracia:.4f}")
    print(f"Precisão: {precisao:.4f}")
    print(f"Recall (Revocação): {revocacao:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("\n" + "="*50 + "\n")

# Comparação e Conclusão sobre a minimização de Falsos Negativos

df_resultados_sensor = pd.DataFrame(resultados_sensor).T

print("Comparação de Desempenho dos Modelos:")
print(df_resultados_sensor)

print("\n----------------- Objetivo Principal e Métrica Chave -----------------")

# Objetivo: detectar todos os sensores defeituosos (Minimizar Falsos Negativos - FN).
print("\nO objetivo é MINIMIZAR FALSOS NEGATIVOS (FN), ou seja, não deixar defeituosos passarem.")

# A métrica que foca na redução de FN.
print("\nA MELHOR MÉTRICA para este objetivo é o RECALL - REVOCAÇÃO.")

print("\nExplicação:")
# Recall = VP / (VP + FN). Mede a capacidade de encontrar todos os positivos reais.
print("O **Recall** mede quantos dos sensores REALMENTE defeituosos foram CORRETAMENTE identificados.")
print("Um Recall alto significa poucos Falsos Negativos (baixo risco de defeito chegar ao cliente).")

# Análise Baseada nos Resultados:
melhor_recall = df_resultados_sensor['Recall'].idxmax()
print(f"\nO modelo com melhor Recall é **{melhor_recall}**, com **{df_resultados_sensor.loc[melhor_recall, 'Recall']:.4f}**.")
print(f"Isso significa que este modelo capturou {df_resultados_sensor.loc[melhor_recall, 'Recall']:.2%} de todos os defeitos reais.")